# Import Libraries

In [1]:
import os
import numpy as np
import cv2
from PIL import Image
import xml.etree.ElementTree as ET
from pascal_voc_writer import Writer

print('✔️ Libraries Imported Successfully!')

✔️ Libraries Imported Successfully!


## Define paths

In [2]:
paths = {
    'workspace_path': os.path.join('Tensorflow', 'workspace'),
    'scripts_path': os.path.join('Tensorflow','scripts'),
    'apimodel_path': os.path.join('Tensorflow','models'),
    'annotation_path': os.path.join('Tensorflow', 'workspace','annotations'),
    'image_path': os.path.join('Tensorflow', 'workspace','images'),
    'model_path': os.path.join('Tensorflow', 'workspace','models'),
    'pretrained_model_path': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
 }

In [6]:
image_folder_path = os.path.join('Tensorflow', 'workspace','images')
train_folder_path = os.path.join(image_folder_path,'train') # same as "Tensorflow\\workspace\\images\\train"
validation_folder_path = os.path.join(image_folder_path,'validation')
test_folder_path = os.path.join(image_folder_path,'test')

## Total number of files in folders

In [9]:
# os.listdir returns list containing names of all the files and folders present in a directory.
# In our example train,validation and test folder contains only images and their xml annotations.
file_list_train = os.listdir(train_folder_path)

# printing first 5 elements of file_list_train
file_list_train[:5]

['10PlateBaza275.jpg',
 '10PlateBaza275.xml',
 '10PlateBaza299.jpg',
 '10PlateBaza299.xml',
 '10xemay112.jpg']

In [19]:
def num_files(folder_path):
    '''
    Takes folder path as input and prints the number of files in a folder
    Note- subfolders and their files are not considered while returning result.
    '''
    
    # list of files name in a folder
    file_list = [file for file in os.listdir(folder_path) 
         if os.path.isfile(os.path.join(folder_path, file))] #sub folders names and files in them are not included
    # folder_name derivied by splitting folder_path on("\\") and take the last element of the split list 
    folder_name = folder_path.split("\\")[-1]
    
    # printing total number of file in a folder list by using len function.
    print(f"Total files in {folder_name} folder : ", len(file_list))
    
print('✔️ Function Defined!')

✔️ Function Defined!


In [20]:
num_files(train_folder_path)

Total files in train folder :  5075


In [23]:
num_files(validation_folder_path)

Total files in validation folder :  1091


In [24]:
num_files(test_folder_path)

Total files in test folder :  1091


## Types of file formats in train and test folder and their count

In [31]:
# function to get unique file extensions and their counts
def unique_ext(folder_path):
    '''
    Take folder path as input and prints the unique file extensions present
    in the folder along with their counts.
    '''
    
    # list of files name in a folder
    file_list = [file for file in os.listdir(folder_path) 
         if os.path.isfile(os.path.join(folder_path, file))] #sub folders names not included
    
    # Creating a list of extensions of files present in a folder,
    # first by splitting the name of files on "." then take the last element "[-1]" of split as extension of file.
    extensions_list = [ext.split('.')[-1] for ext in file_list]
    
    #convert extension list to numpy array
    x = np.array(extensions_list) 
    
    # using np.unique to return unique extensions and their counts in a numpy array
    unique, counts = np.unique(x, return_counts=True) 
    
    # folder_name derivied by splitting folder_path on("\\") and take the last element of the split list 
    folder_name = folder_path.split("\\")[-1]
    
    # print numpy array
    print (f'Number of unique extensions in {folder_name} folder : \n',np.asarray((unique, counts)).T)

print('✔️ Function Defined!')

✔️ Function Defined!


In [32]:
unique_ext(train_folder_path)

Number of unique extensions in train folder : 
 [['ini' '1']
 ['jpg' '2229']
 ['png' '308']
 ['xml' '2537']]


In [33]:
unique_ext(validation_folder_path)

Number of unique extensions in validation folder : 
 [['ini' '1']
 ['jpg' '479']
 ['png' '66']
 ['xml' '545']]


In [34]:
unique_ext(test_folder_path)

Number of unique extensions in test folder : 
 [['ini' '1']
 ['jpg' '479']
 ['png' '66']
 ['xml' '545']]


## Convert jpeg/png files into jpg

In [35]:
def img_to_jpg(folder_path):
    '''
    Converts and save images of JPEG and PNG format to JPG.
    Delete the JPEG and PNG format images
    Also modify and save their respective xml annotation files
    '''
    
    # list of file names in a folder
    file_list = [file for file in os.listdir(folder_path) 
         if os.path.isfile(os.path.join(folder_path, file))]
    
    # image formats to convert in jpg
    img_extensions = ['jpeg','png']
    count = 0    
    for file in file_list:
        file_name = file.split('.')[:-1]
        file_name_str = '.'.join([str(elem) for elem in file_name])
        file_name_jpg = file_name_str + '.jpg'
        file_name_xml = file_name_str + '.xml'
        file_ext = file.split('.')[-1]

        if file_ext in img_extensions:

            im = Image.open(os.path.join(folder_path,file))

            # converting to jpg
            rgb_im = im.convert("RGB")

            # exporting the image
            rgb_im.save(os.path.join(folder_path,file_name_jpg))
            os.remove(os.path.join(folder_path,file))
            
            # Updating image name in xml annotation file
            mytree = ET.parse(os.path.join(folder_path,file_name_xml))
            myroot = mytree.getroot()

            
            # iterating through the name values.
            for names in myroot.iter('filename'):
                
                # updates the name value
                names.text = str(file_name_jpg)

            
            mytree.write(os.path.join(folder_path,file_name_xml))
            count+=1
            
    print('Number of files converted : ', count)


print('✔️ Function Defined!')

✔️ Function Defined!


In [36]:
# testing function
folder_path = "Tensorflow\\workspace\\images\\new"

unique_ext(folder_path)

Number of unique extensions in new folder : 
 [['jpg' '2']
 ['png' '3']
 ['xml' '5']]


In [37]:
img_to_jpg(folder_path)

Number of files converted :  3


In [38]:
unique_ext(folder_path)

Number of unique extensions in new folder : 
 [['jpg' '5']
 ['xml' '5']]


## Bifurcation of image dimensions in a folder

In [44]:
def images_size_in_folder(folder_path, lower_image_size = 300, upper_image_size = 512):
    '''
    Takes folder_path, lower_image_size, upper_image_size as inputs,
    it reads image file of extensions in img_extensions from folder_path and take minimum dimension(height or width)
    of an image and store in a list min_dimension. Later we use lower_image_size and upper_image_size values
    to count the following results
    prints following output - 
    1. Size of images in folder :
    2. Number of images with dimension < lower_image_size
    3. Number of images with dimension >= lower_image_size and <= upper_image_size
    4. Number of images with dimension > upper_image_size
    '''
    # list of files name in a folder
    file_list = [file for file in os.listdir(folder_path) 
         if os.path.isfile(os.path.join(folder_path, file))] #sub folders names not included
    
    # min_dimension lists for storing min dimension(height or width) of images.
    min_dimension = []
    
    img_extensions = ['jpg','jpeg','png']
    
    for file in file_list:
        
        file_ext = file.split('.')[-1]
        
        if file_ext in img_extensions:

            # read image in open cv
            im = cv2.imread(os.path.join(folder_path,file))
            
            img_size = im.shape[:2] # img_size is in (height, width) format
            
            # append min dimension(height or width) of image in min_dimension list
            min_dimension.append(min(img_size))
            
    # folder_name derivied by splitting folder_path on("\\") and take the last element of the split list 
    folder_name = folder_path.split("\\")[-1]
    print(f'Size of images in {folder_name} folder :')
    print(f'Number of images with dimension < {lower_image_size}pxl : ', len([i for i in min_dimension if i < lower_image_size]))
    print(f'Number of images with dimension >= {lower_image_size}pxl and <= {upper_image_size}pxl : ', len([i for i in min_dimension if (i >= lower_image_size) & (i <= upper_image_size)]))
    print(f'Number of images with dimension > {upper_image_size}pxl : ', len([i for i in min_dimension if i > upper_image_size]))

print('✔️ Function Defined!')

✔️ Function Defined!


In [46]:
images_size_in_folder(train_folder_path)
images_size_in_folder(validation_folder_path)
images_size_in_folder(test_folder_path,250,300)

Size of images in train folder :
Number of images with dimension < 300pxl :  2537
Number of images with dimension >= 300pxl and <= 512pxl :  0
Number of images with dimension > 512pxl :  0
Size of images in validation folder :
Number of images with dimension < 300pxl :  545
Number of images with dimension >= 300pxl and <= 512pxl :  0
Number of images with dimension > 512pxl :  0
Size of images in test folder :
Number of images with dimension < 250pxl :  0
Number of images with dimension >= 250pxl and <= 300pxl :  545
Number of images with dimension > 300pxl :  0


In [52]:
paths

{'potrait': 'Tensorflow\\workspace\\images\\new\\potrait_images',
 'Img_512+': 'Tensorflow\\workspace\\images\\new\\Img_512+',
 'Img_300_to_512)': 'Tensorflow\\workspace\\images\\new\\Img_300_to_512)',
 'Img_300-': 'Tensorflow\\workspace\\images\\new\\Img_300-'}

## Image file segregation by image dimension
This function later comes handy while doing Image Augmentation using Albumentations

In [71]:
def image_file_segregation_by_dimension(folder_path, lower_image_size = 300, upper_image_size = 512, potrait = False):
    '''
    Segregate image files and their xml annotation files into different folders based on their size(dimension)
    Takes following input
    folder_path - path of folder containing image files and their xml annotation files on which segregation is to be performed
    lower_image_size - lower range of pixel value of image, default set to 300pxl
    upper_image_size - upper range of pixel value of image, default set to 512pxl
    potrait - set this to 'True' if potrait images are to be segregated, default set to 'False'
    
    Output - 
    create different folders in the parent folder and move files in respective folders according to their pixel size
    create following folders-
    'potrait': if potrair set to 'True'
    'Img_512+'
    'Img_300_to_512)'
    'Img_300-'
    '''
    # list of files name in a folder
    file_list = [file for file in os.listdir(folder_path) 
         if os.path.isfile(os.path.join(folder_path, file))] #sub folders names not included
    
    if potrait ==False:
        
        paths = {
            f'Img_{upper_image_size}+' : os.path.join(folder_path,f'Img_{upper_image_size}+'),
            f'Img_{lower_image_size}_to_{upper_image_size})' : os.path.join(folder_path,f'Img_{lower_image_size}_to_{upper_image_size})'),
            f'Img_{lower_image_size}-' : os.path.join(folder_path,f'Img_{lower_image_size}-')
        }
    else:
        paths = {
            'potrait' : os.path.join(folder_path,'potrait_images'),
            f'Img_{upper_image_size}+' : os.path.join(folder_path,f'Img_{upper_image_size}+'),
            f'Img_{lower_image_size}_to_{upper_image_size}' : os.path.join(folder_path,f'Img_{lower_image_size}_to_{upper_image_size}'),
            f'Img_{lower_image_size}-' : os.path.join(folder_path,f'Img_{lower_image_size}-')
        }

    for path in paths.values():
        if not os.path.exists(path):
            if os.name == 'posix':
                !mkdir -p {path}
            if os.name == 'nt':
                !mkdir {path}

    img_extensions = ['jpg','jpeg','png','JPG','JPEG','PNG']
    
    
    for file in file_list:
        
        file_name = file.split('.')[:-1]
        file_name_str = '.'.join([str(elem) for elem in file_name])
        file_name_xml = file_name_str + '.xml'
        file_ext = file.split('.')[-1]
        
        if file_ext in img_extensions:
            
            # read image in open cv
            im = cv2.imread(os.path.join(folder_path,file))
            # im.shape is in (height, width) format
            height = im.shape[0]
            width = im.shape[1]
            
            # source filepath of image file
            existing_filepath_img = os.path.join(folder_path,file)
            # source filepath of corresponding xml file
            existing_filepath_xml = os.path.join(folder_path,file_name_xml)

            # if image and its xml file is present in folder
            if (os.path.exists(existing_filepath_img)) & (os.path.exists(existing_filepath_xml)):

                if (width/height < 1) & (potrait == True) :
                    # destination filepath of image file
                    new_filepath_img = os.path.join(paths['potrait'],file)
                    # destination filepath of corresponding xml file
                    new_filepath_xml = os.path.join(paths['potrait'],file_name_xml)
                    
                    # moving image file from souce filepath to destination filepath
                    os.replace(existing_filepath_img, new_filepath_img)
                    # moving xml file from souce filepath to destination filepath
                    os.replace(existing_filepath_xml, new_filepath_xml)

                elif (height > upper_image_size) & (width > upper_image_size):

                    new_filepath_img = os.path.join(paths[f'Img_{upper_image_size}+'],file)
                    new_filepath_xml = os.path.join(paths[f'Img_{upper_image_size}+'],file_name_xml)
                    os.replace(existing_filepath_img, new_filepath_img)
                    os.replace(existing_filepath_xml, new_filepath_xml)

                elif ((height >= lower_image_size) & (width >= lower_image_size)):
                    new_filepath_img = os.path.join(paths[f'Img_{lower_image_size}_to_{upper_image_size}'],file)
                    new_filepath_xml = os.path.join(paths[f'Img_{lower_image_size}_to_{upper_image_size}'],file_name_xml)
                    os.replace(existing_filepath_img, new_filepath_img)
                    os.replace(existing_filepath_xml, new_filepath_xml)

                else:
                    new_filepath_img = os.path.join(paths[f'Img_{lower_image_size}-'],file)
                    new_filepath_xml = os.path.join(paths[f'Img_{lower_image_size}-'],file_name_xml)
                    os.replace(existing_filepath_img, new_filepath_img)
                    os.replace(existing_filepath_xml, new_filepath_xml)

    print('✔️ Transfer of files successful')

    
print('✔️ Function Defined!')

✔️ Function Defined!


In [65]:
# testing function
%pwd

'C:\\Users\\gs\\Documents\\python\\anpr'

In [66]:
folder_path = os.path.join('Tensorflow','workspace','images','number_plate_detection','train')

In [67]:
unique_ext(folder_path)

Number of unique extensions in train folder : 
 [['JPG' '1']
 ['ini' '1']
 ['jpg' '1443']
 ['xml' '1444']]


ini is 'desktop.ini' (windows) file in the folder

In [68]:
# print image size bifurcation in train folder 
images_size_in_folder(folder_path)

Size of images in train folder :
Number of images with dimension < 300pxl :  643
Number of images with dimension >= 300pxl and <= 512pxl :  285
Number of images with dimension > 512pxl :  515


In [72]:
# segregate all the images and their xml annotation files in their respective folders
image_file_segregation_by_dimension(folder_path,potrait=True)

✔️ Transfer of files successful


In [13]:
image_file_segregation_by_pxl(test_folder_path)

Transfer of files successful


In [14]:
image_file_segregation_by_pxl(validation_folder_path)

Transfer of files successful


In [15]:
image_file_segregation_by_pxl(train_folder_path)

Transfer of files successful


list

## 2. Image Augmentation using Albumentations

pip install -U albumentations

pip install pascal-voc-writer

In [16]:
import albumentations as A
import cv2
from pascal_voc_writer import Writer
import xml.etree.ElementTree as ET

print('✔️ Libraries Imported Successfully!')

In [17]:
transform = A.Compose([
    A.RandomCrop(width=512, height=512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.RandomGamma(p=0.2), 
    A.RGBShift(p=0.2),
], bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['class_labels']))

In [18]:
def albumentation_pipeline(source_file_path,destination_file_path,num_images):
    
    file_list = [file for file in os.listdir(source_file_path) 
             if os.path.isfile(os.path.join(source_file_path, file))]
    # file_list = os.listdir(source_file_path)
    print('Number of images : ',len(file_list)//2)
    blank_file_count = 0
    file_count = 0
    for file in file_list:

        file_name = file.split('.')[0]
        file_ext = file.split('.')[1]

        if (file_ext == 'jpg'):
            # print((os.path.join(source_file_path,file_name,f'.jpg'))
            img = cv2.imread(os.path.join(source_file_path,f'{file_name}.jpg'))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


            xml_file_address = os.path.join(source_file_path,f'{file_name}.xml')
            mytree = ET.parse(xml_file_address)

            myroot = mytree.getroot()

            # iterating through the bndbox values.
            bbox = []
            class_label = []
            for box in myroot.iter('bndbox'):
                bbox_temp = []
                bbox_temp.append(int(box.find('xmin').text))
                bbox_temp.append(int(box.find('ymin').text))
                bbox_temp.append(int(box.find('xmax').text))
                bbox_temp.append(int(box.find('ymax').text))
                bbox_temp.append('NumberPlate')
                bbox.append(bbox_temp)
                class_label.append('NumberPlate')

            # print(bbox)

            for num_img in range(num_images):
                transformed = transform(image=img, bboxes=bbox, class_labels = class_label)
                transformed_image = transformed['image']
                transformed_bboxes = transformed['bboxes']
                # transformed_class_labels = transformed['class_labels']
                if ((len(transformed_bboxes) == 0) & (blank_file_count < (len(file_list)*.50))) | (len(transformed_bboxes) != 0):

                    
                    cv2.imwrite(os.path.join(destination_file_path, f'{file_name}_{num_img}.jpg' ), transformed_image)
                    # create pascal voc writer (image_path, width, height)
                    # print(transformed_image.shape)
                    writer = Writer(os.path.join(destination_file_path, f'{file_name}_{num_img}.jpg'), 
                                    transformed_image.shape[1], transformed_image.shape[0])
                    # add objects (class, xmin, ymin, xmax, ymax)
                    for info in (transformed_bboxes):
                        writer.addObject(info[4], round(info[0]), round(info[1]), round(info[2]), round(info[3]))
                    # write to file
                    writer.save(os.path.join(destination_file_path, f'{file_name}_{num_img}.xml'))
                    blank_file_count+=1
                    file_count+=1
                    # print(transformed_bboxes)

                else:
                    continue
    print('Total number of augmented images : ', file_count)

In [20]:
source_file_path = os.path.join(paths['image_path'],'train', 'img_512+')
destination_file_path = os.path.join(paths['image_path'],'train','img_512+','aug')
num_images = 20
albumentation_pipeline(source_file_path,destination_file_path,num_images)

Number of images :  260
Total number of augmented images :  3139


In [22]:
source_file_path = os.path.join(paths['image_path'],'validation', 'img_512+')
destination_file_path = os.path.join(paths['image_path'],'validation','img_512+','aug')
num_images = 20
albumentation_pipeline(source_file_path,destination_file_path,num_images)

Number of images :  62
Total number of augmented images :  780


In [16]:
source_file_path = os.path.join(paths['image_path'],'test', 'img_512+')
destination_file_path = os.path.join(paths['image_path'],'test','img_512+','aug')
num_images = 15
albumentation_pipeline(source_file_path,destination_file_path,num_images)

Number of images :  96
Total number of augmented images :  744


In [23]:
transform = A.Compose([
    A.RandomCrop(width=300, height=300),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.RandomGamma(p=0.2), 
    A.RGBShift(p=0.2),
], bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['class_labels']))

In [24]:
source_file_path = os.path.join(paths['image_path'],'train', 'img_300_to_512')
destination_file_path = os.path.join(paths['image_path'],'train','img_300_to_512','aug')
num_images = 20
albumentation_pipeline(source_file_path,destination_file_path,num_images)

Number of images :  179
Total number of augmented images :  3313


In [25]:
source_file_path = os.path.join(paths['image_path'],'validation', 'img_300_to_512')
destination_file_path = os.path.join(paths['image_path'],'validation','img_300_to_512','aug')
num_images = 20
albumentation_pipeline(source_file_path,destination_file_path,num_images)

Number of images :  39
Total number of augmented images :  641


In [19]:
source_file_path = os.path.join(paths['image_path'],'test', 'img_300_to_512')
destination_file_path = os.path.join(paths['image_path'],'test','img_300_to_512','aug')
num_images = 15
albumentation_pipeline(source_file_path,destination_file_path,num_images)

Number of images :  41
Total number of augmented images :  502


## resizing images alongwith annotations while maintaining aspect ratio

In [7]:
im_path = "Tensorflow/workspace/images/resize/License (25).png"

In [14]:
im = cv2.imread(im_path)

In [10]:
(new_height,new_width) = im.shape[:2]

In [12]:
im.shape[:2][0]

36

In [ ]:
file_list = [file for file in os.listdir(source_file_path) 
             if os.path.isfile(os.path.join(source_file_path, file))]
    # file_list = os.listdir(source_file_path)
    print('Number of images : ',len(file_list)//2)
    # blank_file_count = 0
    # file_count = 0
    for file in file_list:

        file_name = file.split('.')[0]
        file_ext = file.split('.')[1]

        if (file_ext == 'png'):
            # print((os.path.join(source_file_path,file_name,f'.jpg'))
            img = cv2.imread(os.path.join(source_file_path,f'{file_name}.png'))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


            xml_file_address = os.path.join(source_file_path,f'{file_name}.xml')
            xml_list =[]
            tree = ET.parse(xml_file_address)
            root = tree.getroot()
            for member in root.findall('object'):
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[4].find('xmin').text),
                         int(member[4].find('ymin').text),
                         int(member[4].find('xmax').text),
                         int(member[4].find('ymax').text)
                         )
                xml_list.append(value)
                    
                cv2.imwrite(os.path.join(destination_file_path, f'{file_name}.jpg' ), transformed_image)
                # create pascal voc writer (image_path, width, height)
                # print(transformed_image.shape)
                writer = Writer(os.path.join(destination_file_path, f'{file_name}.jpg'), 
                                transformed_image.shape[1], transformed_image.shape[0])
                # add objects (class, xmin, ymin, xmax, ymax)
                for info in (transformed_bboxes):
                    writer.addObject(info[4], round(info[0]), round(info[1]), round(info[2]), round(info[3]))
                # write to file
                writer.save(os.path.join(destination_file_path, f'{file_name}.xml'))

In [8]:
def image_resize(source_file_path, destination_file_path , desired_size, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    file_list = [file for file in os.listdir(source_file_path) 
             if os.path.isfile(os.path.join(source_file_path, file))]
    # file_list = os.listdir(source_file_path)
    print('Number of images : ',len(file_list)//2)
    # blank_file_count = 0
    # file_count = 0
    for file in file_list:

        file_name = file.split('.')[0]
        file_ext = file.split('.')[1]

        if ((file_ext == 'png') | (file_ext == 'jpg')):
            # print((os.path.join(source_file_path,file_name,f'.jpg'))
            image = cv2.imread(os.path.join(source_file_path,f'{file}'))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


            xml_file_address = os.path.join(source_file_path,f'{file_name}.xml')
            
            xml_list =[]
            tree = ET.parse(xml_file_address)
            root = tree.getroot()
            for member in root.findall('object'):
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[4].find('xmin').text),
                         int(member[4].find('ymin').text),
                         int(member[4].find('xmax').text),
                         int(member[4].find('ymax').text)
                         )
                xml_list.append(value)
            old_size = image.shape[:2]
            ratio = float(desired_size)/max(old_size)
            new_size = tuple([int(x*ratio) for x in old_size])
            
#             dim = None
            

#             # if both the width and height are None, then return the
#             # original image
#             if width is None and height is None:
#                 return image

#             # check to see if the width is None
#             if width is None:
#                 # calculate the ratio of the height and construct the
#                 # dimensions
#                 r = height / float(h)
#                 dim = (int(w * r), height)

#             # otherwise, the height is None
#             else:
#                 # calculate the ratio of the width and construct the
#                 # dimensions
#                 r = width / float(w)
#                 dim = (width, int(h * r))

            # resize the image
            resized = cv2.resize(image, (new_size[1],new_size[0]))

            # new_size = resized.shape[:2]
            # print(new_size)
            # delta_w = resized.shape[:2][1] -  image.shape[:2][1]
            # delta_h = resized.shape[:2][0] -  image/shape[:2][0]

            delta_w = desired_size - new_size[1]
            delta_h = desired_size - new_size[0]
            top, bottom = delta_h//2, delta_h-(delta_h//2)
            left, right = delta_w//2, delta_w-(delta_w//2)
            # print(top,bottom,left,right)

            color = [255, 255, 255]
            new_im = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
            cv2.imwrite(os.path.join(destination_file_path, f'{file}' ), new_im)
            
            writer = Writer(os.path.join(destination_file_path, f'{file}'), 
                                new_im.shape[1], new_im.shape[0])
            # add objects (class, xmin, ymin, xmax, ymax)
            for info in (xml_list):
                writer.addObject(info[3], round(info[4]*ratio + left), round(info[5]*ratio + bottom), round(info[6]*ratio + right), round(info[7]*ratio + top))
            # write to file
            writer.save(os.path.join(destination_file_path, f'{file_name}.xml'))
    
    


    # return the resized image
    # return resized

In [9]:
source_file_path = os.path.join('number_plate_digit_dataset')
destination_file_path = os.path.join('data_1')

In [10]:
image_resize(source_file_path,destination_file_path,256,256)

Number of images :  3627


In [66]:
round(34)

34

In [29]:
xml_path = "Tensorflow/workspace/images/resize/License (25).xml"
xml_list =[]
tree = ET.parse(xml_path)
root = tree.getroot()
for member in root.findall('object'):
    value = (root.find('filename').text,
             int(root.find('size')[0].text),
             int(root.find('size')[1].text),
             member[0].text,
             int(member[4].find('xmin').text),
             int(member[4].find('ymin').text),
             int(member[4].find('xmax').text),
             int(member[4].find('ymax').text)
             )
    xml_list.append(value)

In [30]:
xml_list

[('License (25).png', 128, 36, 'W', 14, 7, 23, 25),
 ('License (25).png', 128, 36, 'B', 26, 7, 34, 25),
 ('License (25).png', 128, 36, '0', 37, 8, 45, 25),
 ('License (25).png', 128, 36, '2', 48, 8, 57, 25),
 ('License (25).png', 128, 36, 'A', 58, 7, 67, 25),
 ('License (25).png', 128, 36, 'A', 69, 8, 77, 25),
 ('License (25).png', 128, 36, '3', 80, 8, 89, 25),
 ('License (25).png', 128, 36, '5', 91, 8, 100, 25),
 ('License (25).png', 128, 36, '7', 101, 8, 110, 25),
 ('License (25).png', 128, 36, '2', 113, 8, 122, 25)]

In [31]:
xml_list[1]

('License (25).png', 128, 36, 'B', 26, 7, 34, 25)

In [32]:
 for info in (xml_list):
        print(info[7])

25
25
25
25
25
25
25
25
25
25
